In [20]:
#from Bio.Seq import Seq
#from Bio import SeqIO
import re
import os
import operator
import collections
from collections import OrderedDict
from sys import exit

def main():
    fout = open('snp_reference.txt', 'wt')
    #for seq_record in SeqIO.parse('HSGRCH3771_final.fa', 'fasta'):
    #    get_loci(seq_record.id, seq_record.seq, fout)
    ref_genome = genomeparse('nipponbare7.fa')
    
    for chrid, chrseq in ref_genome.items():
        if not re.search(r'Chr(\d+)', chrid):
            break
        else:
            get_loci(chrid, chrseq, fout)
    
    fout.close()
    
    convert_to_vcf()

    
## parse reference genome
def genomeparse(genome, fmt = 'fasta'):
    seq = OrderedDict()
    with open (genome) as f:
        chm = ''
        sequence = ''
        for line in f:
            line = line.rstrip().replace(' ', '')
            if line.startswith('>'):
                seq[chm] = sequence
                sequence = ''
                chm = re.sub('>', '', line)
                #print (chm)
                continue
            sequence += line
        seq[chm] = sequence
        del seq['']
    return seq    
 
    
## get the allele in reference    
def get_loci(chrid, chrseq, fout):
    with open('maf.txt', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split()
            loci = lst[0].split('-')
            if re.search(r'chr0', loci[0]):
                ch = re.sub(r'chr0', r'Chr', loci[0])
            else: 
                ch = re.sub(r'c', r'C', loci[0])
                     
            if ch == chrid:
                allele = chrseq[int(loci[1])-1]
                #print (ch+'\t'+loci[1]+'\t'+allele+'\n')
                fout.write(ch+'\t'+loci[1]+'\t'+allele+'\n')

                
## convert to vcf             
def convert_to_vcf():
    nts = ['A', 'T', 'C', 'G']
    ref = OrderedDict()

    fh = open('converted_snp.vcf', 'wt')
    fh.write('##fileformat=VCFv4.2\n')
    fh.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t')

    with open('nt_order.txt', 'rt') as f:
        for line in f:
            line = line.rstrip()
            fh.write(line+'\t')

    fh.write('\n')

    with open('snp_reference.txt', 'rt') as f:
        for line in f:
            lst = line.rstrip().split()
            loci = lst[0]+'\t'+lst[1]
            ref[loci] = lst[2]
        
    with open('maf.txt', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split()
            pos = lst[0].split('-')
            if re.search(r'chr0', pos[0]):
                pos[0] = re.sub(r'chr0', r'Chr', pos[0])
            else: 
                pos[0] = re.sub(r'c', r'C', pos[0])
        
            loci = pos[0]+'\t'+pos[1]
            fh.write(pos[0]+'\t'+pos[1]+'\t'+'.\t'+ref[loci]+'\t') #CHROM POS REF
        
            nt = ''
            for i in range(len(lst[1])):
                if lst[1][i] in nts:
                    nt = nt+lst[1][i]
        
            freq = collections.Counter(nt)
            sorted_freq = sorted(freq.items(), key = operator.itemgetter(1), reverse = True)
        
            if sorted_freq[0][0] == ref[loci]:
                fh.write(sorted_freq[1][0]+'\t.\t.\t.\tGT\t') #ALT
            else:
                fh.write(sorted_freq[0][0]+'\t.\t.\t.\tGT\t') #ALT

            for s in lst[1]:
                if s == ref[loci]:
                    fh.write('0/0\t')
                elif s in nts and s != ref[loci]:
                    fh.write('1/1\t')
                elif s == '-':
                    fh.write('./.\t')
                else:
                    fh.write('0/1\t')
            fh.write('\n')
        
    fh.close()      

#os.system('tar -czvf converted_snp.vcf.tar.gz converted_snp.vcf')
os.system('cp converted_snp.vcf converted_snp.vcf.bkup')
os.system('gzip converted_snp.vcf')
                
if __name__ == '__main__':
    main()

In [56]:
### convert to vcf
import collections
from collections import OrderedDict
import operator
import os


nts = ['A', 'T', 'C', 'G']
ref = OrderedDict()

fh = open('converted_snp.vcf', 'wt')
fh.write('##fileformat=VCFv4.2\n')
fh.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t')

with open('nt_order.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        fh.write(line+'\t')

fh.write('\n')

with open('snp_reference.txt', 'rt') as f:
    for line in f:
        lst = line.rstrip().split()
        loci = lst[0]+'\t'+lst[1]
        ref[loci] = lst[2]
        
with open('maf.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split()
        pos = lst[0].split('-')
        if re.search(r'chr0', pos[0]):
            pos[0] = re.sub(r'chr0', r'Chr', pos[0])
        else: 
            pos[0] = re.sub(r'c', r'C', pos[0])
        
        loci = pos[0]+'\t'+pos[1]
        fh.write(pos[0]+'\t'+pos[1]+'\t'+'.\t'+ref[loci]+'\t') #CHROM POS REF
        
        nt = ''
        for i in range(len(lst[1])):
            if lst[1][i] in nts:
                nt = nt+lst[1][i]
        
        freq = collections.Counter(nt)
        sorted_freq = sorted(freq.items(), key = operator.itemgetter(1), reverse = True)
        
        if sorted_freq[0][0] == ref[loci]:
            fh.write(sorted_freq[1][0]+'\t.\t.\t.\tGT\t') #ALT
        else:
            fh.write(sorted_freq[0][0]+'\t.\t.\t.\tGT\t') #ALT

        for s in lst[1]:
            if s == ref[loci]:
                fh.write('0/0\t')
            elif s in nts and s != ref[loci]:
                fh.write('1/1\t')
            elif s == '-':
                fh.write('./.\t')
            else:
                fh.write('0/1\t')
        fh.write('\n')
        
fh.close()      

#os.system('tar -czvf converted_snp.vcf.tar.gz converted_snp.vcf')
os.system('cp converted_snp.vcf converted_snp.vcf.bkup')
os.system('gzip converted_snp.vcf')

In [22]:
lst = [1,2]
if lst[1] < 3:
    lst[1] = 3

print (lst)

[1, 3]


In [27]:
y = 'TTCGTG'
nt = ['A', 'T', 'C', 'G']
for i in nt:
    if i not in y:
        print (i)
        
for j in y:
    print (j)

A
T
T
C
G
T
G


In [47]:
import collections
import operator
loc = 'abcadebb'
fre = collections.Counter(loc)
print (fre)
sorted_fre = sorted(fre.items(), key = operator.itemgetter(1), reverse=True)
print (sorted_fre[0][0])

Counter({'b': 3, 'a': 2, 'e': 1, 'd': 1, 'c': 1})
b


In [50]:
n = 'abc'
for x in ['d','e','f']:
    n = n+x
    print (n)

abcd
abcde
abcdef


In [13]:
#### test

import re
def get_loci(chrid, chrseq):
    with open('test.snp', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split()
            loci = lst[0].split('-')
            if re.search(r'chr0', loci[0]):
                ch = re.sub(r'chr0', r'Chr', loci[0])
            else: 
                ch = re.sub(r'c', r'C', loci[0])
            
            if ch == chrid:
                allele = chrseq[int(loci[1])-1]
                print (ch+'\t'+loci[1]+'\t'+allele+'\n')
                #fout.write(ch+'\t'+loci[1]+'\t'+allele+'\n')
                
chrid = 'Chr2'
chrseq = 'ATCGTGACGTGCTGCGTGGGAGCAGCT'
get_loci(chrid, chrseq)

Chr2	7	A



In [14]:
for i in range(6):
    if i < 5:
        print(i)
    else:
        break

0
1
2
3
4
